### 매출에 영향력 높은 후행 지표 찾기

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
base_path = "C:/Users/dukkubi"
df = pd.read_csv(base_path + "영업팀 매출 데이터.csv")
form = pd.read_csv(base_path + "피터팬의 좋은방구하기 입점문의 접수(응답).csv")
agent = pd.read_csv(base_path + "중개사 데이터.csv")
first_party_db = pd.read_csv(base_path + "first_party_db.csv")

##### 예시 데이터로 대체

In [83]:
import pandas as pd
import numpy as np

# df 데이터프레임 생성
data = {
    '지역': [f'지역{i%5}' for i in range(100)],
    '상호': [f'상호{i%10}' for i in range(100)],
    '대표자': [f'대표자{i%7}' for i in range(100)],
    '매출액': [f'{np.random.randint(10, 500) * 1000:,}' for _ in range(100)],
    '신규연장': np.random.choice(['신규', '연장'], size=100, p=[0.5, 0.5]),
    '날짜': pd.date_range(start='2023-01-01', periods=100).strftime('%Y-%m-%d')
}
df = pd.DataFrame(data)
df['지역 상호 대표자'] = df['지역'] + ' ' + df['상호'] + ' ' + df['대표자']

# first_party_db 데이터프레임 생성
unique_entries = df['지역 상호 대표자'].unique()[:90]  # 일부만 포함시켜 삭제되는 행이 적도록 설계
first_party_data = {
    '지역 상호 대표자': unique_entries,
    'register_number': [f'RN{str(i).zfill(3)}' for i in range(len(unique_entries))]
}
first_party_db = pd.DataFrame(first_party_data)

영업팀 매출 데이터에 중개사를 식별하는 key가 존재하지 않는다.<br>
따라서, 지역 + 상호 + 대표자 데이터를 공공 데이터의 지역 + 상호 + 대표자와 join 하여 등록번호를 부여했다.

In [84]:
df['지역 상호 대표자'] = df['지역']  + ' ' + df['상호'] + ' ' + df['대표자']

first_party_db['지역 상호 대표자'] = df['지역']  + ' ' + df['상호'] + ' ' + df['대표자']
first_party_db = first_party_db[['지역 상호 대표자', 'register_number']]

df = df.merge(first_party_db, on='지역 상호 대표자', how='left')
df = df.dropna()
df

,지역,상호,대표자,매출액,신규연장,날짜,지역 상호 대표자,register_number
0,지역0,상호0,대표자0,"408,000",신규,2023-01-01,지역0 상호0 대표자0,RN000
1,지역1,상호1,대표자1,"391,000",신규,2023-01-02,지역1 상호1 대표자1,RN001
2,지역2,상호2,대표자2,"436,000",신규,2023-01-03,지역2 상호2 대표자2,RN002
3,지역3,상호3,대표자3,"388,000",연장,2023-01-04,지역3 상호3 대표자3,RN003
4,지역4,상호4,대표자4,"489,000",신규,2023-01-05,지역4 상호4 대표자4,RN004
...,...,...,...,...,...,...,...,...
95,지역0,상호5,대표자4,"343,000",연장,2023-04-06,지역0 상호5 대표자4,RN025
96,지역1,상호6,대표자5,"422,000",연장,2023-04-07,지역1 상호6 대표자5,RN026
97,지역2,상호7,대표자6,"247,000",연장,2023-04-08,지역2 상호7 대표자6,RN027
98,지역3,상호8,대표자0,"28,000",신규,2023-04-09,지역3 상호8 대표자0,RN028


#### 전처리

In [85]:
# '매출액' 열의 빈 문자열을 0으로 대체하고 '-' 기호 제거하고 쉼표(,) 제거한 후에 숫자형으로 변환
df['매출액'] = df['매출액'].replace('', '0').str.replace(',', '').astype(int)

# '신규연장' 컬럼이 null 값인 경우, '연장'으로 대체
df['신규연장'] = df['신규연장'].fillna('연장')

#### 신규가입 중개사 vs 재계약(연장) 중개사 비교

In [86]:
# 신규 가입 중개사 수와 재개약(연장) 중개사 수 비교 
print(df.groupby('신규연장', dropna=False).size().reset_index(name='count'))

# 신규 가입 중개사의 매출과 연장 중개사의 매출 비교
print(df.groupby('신규연장', dropna=False)['매출액'].sum().reset_index())

  신규연장  count
0   신규     58
1   연장     42
  신규연장       매출액
0   신규  15593000
1   연장  11729000


연장 중개사 수, 매출 비율이 더 높은 점을 확인했다.<br>
평균적으로 중개사가 몇 개월 단위로, 몇 회 연장을 하는지도 확인을 해야한다.<br>
<br>
ex) 1개월 당 꾸준히 100명의 신규 중개사 유입
- 평균 연장 횟수: 3회
- 1회 연장 당 평균적으로 걸리는 기간: 3개월<br>

위 조건에서 작년 이번 달 처음 100명의 신규 중개사가 유입되었다고 가정하면 다음과 같은 결과가 나온다.

- 1개월 뒤 : 신규 중개사 100명, 연장 중개사 0명
- 2개월 뒤 : 신규 중개사 100명, 연장 중개사 0명
- 3개월 뒤 : 신규 중개사 100명, 연장 중개사 100명
- 4개월 뒤 : 신규 중개사 100명, 연장 중개사 100명
- 5개월 뒤 : 신규 중개사 100명, 연장 중개사 100명
- 6개월 뒤 : 신규 중개사 100명, 연장 중개사 200명
- 7개월 뒤 : 신규 중개사 100명, 연장 중개사 200명
- 8개월 뒤 : 신규 중개사 100명, 연장 중개사 200명
- 9개월 뒤 : 신규 중개사 100명, 연장 중개사 300명
- 10개월 뒤 : 신규 중개사 100명, 연장 중개사 300명
- 11개월 뒤 : 신규 중개사 100명, 연장 중개사 300명
- 12개월 뒤 : 신규 중개사 100명, 연장 중개사 300명

#### 평균 연장 횟수 구하기

In [88]:
from datetime import date
from dateutil.relativedelta import relativedelta

# 중개사별 첫 구매 날짜
first_purchase_date = df.loc[df['신규연장'] == '신규', ['날짜', 'register_number']].rename(columns={'날짜': '신규가입날짜'})

# 월 단위로 변경
first_purchase_date['신규가입날짜'] =  pd.to_datetime(first_purchase_date['신규가입날짜']).dt.strftime('%Y-%m')

# 기존 데이터에 첫 구매 날짜 join
df = df.merge(first_purchase_date, on='register_number', how='left')

# 연장 날짜 - 첫 구매 날짜 계산
df['연장경과개월'] = (
    (pd.to_datetime(df['날짜']).dt.year - pd.to_datetime(df['신규가입날짜'], format='%Y-%m').dt.year) * 12 +
    (pd.to_datetime(df['날짜']).dt.month - pd.to_datetime(df['신규가입날짜'], format='%Y-%m').dt.month)
)

# 첫 구매일로부터 1개월 단위로 연장한 횟수
df_cohort = df.groupby(['신규가입날짜', '연장경과개월'])['register_number'].nunique().reset_index()

# 11개월 경과까지의 데이터만 선택. 12개월치 데이터가 없는 중개사는 선택 X
year_month_of_12_ago = (date.today() - relativedelta(months=12)).strftime('%Y-%m')
condition1 = (df_cohort['연장경과개월'] < 12)
condition2 = (df_cohort['신규가입날짜'] <= year_month_of_12_ago)
df_cohort = df_cohort.loc[condition1 & condition2]

# 평균 연장 횟수
년월기준신규중개사수 = df_cohort.loc[df_cohort['연장경과개월'] == 0].groupby('신규가입날짜')['register_number'].sum().reset_index()['register_number'].sum()
년월기준연장중개사수 = df_cohort.loc[df_cohort['연장경과개월'] != 0].groupby('신규가입날짜')['register_number'].sum().reset_index()['register_number'].sum()
print(년월기준연장중개사수/년월기준신규중개사수)

0.5


#### 1회 연장 당 평균적으로 걸리는 기간 구하기

In [89]:
df.groupby(['날짜', 'register_number'])['신규연장'].size().reset_index()

,날짜,register_number,신규연장
0,2023-01-01,RN000,1
1,2023-01-02,RN001,1
2,2023-01-03,RN002,1
3,2023-01-04,RN003,1
4,2023-01-05,RN004,2
...,...,...,...
95,2023-04-06,RN025,1
96,2023-04-07,RN026,1
97,2023-04-08,RN027,1
98,2023-04-09,RN028,2


#### 신규 가입 후, 3개월 안에 이탈하는 이탈률 구하기

#### 1차 후행 지표 선정

1회 연장 당 평균적으로 걸리는 기간은 업계 특성상 쉽게 바뀔 수 없다.<br>
억지로 기간을 줄이려고 하면 역효과가 나타날 수 있다.<br>
또한, 평균 연장 횟수가 같이 늘어나지 않는 이상 연장 기간이 줄어드는건 수치상 큰 의미는 없다.<br>
따라서, 매출을 늘리기 위해 개선이 필요한 지표는 다음과 같다.

**개선이 필요한 지표**
- 신규 중개사 수
- 평균 연장 횟수
- ARPPU

이를 식으로 대략적으로 나타내면 다음과 같다.

$ 매출 = (신규 중개사 수 + (신규 중개사 수 \times 평균 연장 횟수)) * ARPPU $

신규 중개사 수는 단위가 크지만, 평균 연장 횟수는 단위가 작으므로 1회만 늘려도 큰 효과를 볼 수 있다.<br>
ex) 1개월당 꾸준히 유입되는 신규 중개사가 50명이 넘는 상황에서는 신규 중개사 10명 늘리는 것보다 평균 연장 횟수 1회 늘리는 게 더 큰 효과를 본다.<br>

ARPPU는 영업팀, 경영전략팀에서 전담해서 맡고 있기 때문에 ARPPU에 악영향을 주지 않는 방향으로 나머지 2지표에 집중하기로 했다.

- 평균 연장 횟수는 서비스 팀, 마케팅 팀에서 서포트 가능.
- 신규 중개사 수는 마케팅 팀에서 서포트 가능.

평균 연장 횟수에 기여하는 지표가 무엇인지 조사해보았다.

지금까지는 평균 연장 횟수를 전체 중개사 대상으로 조사했지만, 개별 조사도 필요하다.<br>
평균 연장 횟수의 분산을 통해 상위 20% 그룹, 중간 그룹, 하위 20% 그룹으로 나누었다.

각 그룹 대상으로 다시 지표를 살펴보았다.

#### 유입 채널에 따른 유입수, 전환율 차이